In [1]:
%matplotlib notebook
import numpy as np
from numpy.fft import fft2, ifft2, fftshift
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import scipy
from scipy.optimize import curve_fit
from scipy.ndimage import uniform_filter as uf

from skimage.filters import threshold_otsu, threshold_local
from skimage.transform import downscale_local_mean #For binning

import xarray as xr #package for labeling and adding metadata to multi-dimensional arrays   
import tifffile 

import pandas as pd
import io 
import sys
import csv
import os
import glob #glob is helpful for searching for filenames or directories
import pickle #for saving data
### usually this block prints out "nd2reader module not found. Reading of .nd2 files disabled." on the first run
### this is fine (unless you need to read .nd2 files), just re-run this block to make the error go away

C:\Users\svc-scst291lab\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
### this function (newRadav) finds the radial average of the image autocorrelation in the SIA function 
def newRadav(im, limangles=False, angRange=None, mask=None, rev=False,
             debug_q = None):
    if mask is None:
        hasMask = False
    else:
        hasMask = True
    nx,ny = im.shape
    xx = np.arange(-(nx-1)/2., nx/2.)
    yy = np.arange(-(ny-1)/2., ny/2.)
    #x,y = np.meshgrid(xx,yy)
    x,y = np.meshgrid(yy,xx)
    q = np.sqrt(x**2 + y**2)
    angles = np.arctan2(x,y)
    
    qx = np.arange(-1*nx/2,nx/2)*(1./nx) * max(nx,ny)
    qy = np.arange(-1*ny/2,ny/2)*(1./ny) * max(nx,ny)
    qxx,qyy = np.meshgrid(qy,qx) #qy,qx is correct order
    q_new = np.sqrt(qxx**2 + qyy**2)
    
    if debug_q is not None:
        return q_new.round().astype(int)==debug_q
    
    if mask is None:
        mask = np.ones_like(angles)
    if angRange is not None:
        w1 = np.where(angles>angRange[0])
    else:
        w1 = np.where(angles>(13*np.pi/14))
    if mask is None:
        mask[w1]=0
        mask = mask * np.rot90(np.rot90(mask))
        mask = mask * np.flipud(mask)
        mask[np.where(mask==0)] = np.nan
        if rev:
            mask = np.rot90(mask)
    qr = q_new.round().astype(int)
    #rs = np.arange(0,(nx-1)/2)
    rs = np.arange(0,(max(nx,ny)-1)/2) 
    radav = np.zeros((len(rs)),dtype=float)
    for i in range(0,len(rs)):
        w = np.where(qr==rs[i])
        if len(w[0])>0:
            if limangles or hasMask:
                newim = im*mask
                radav[i] = np.nanmean(newim[w])
            else:
                radav[i] = np.nanmean(im[w])
        #else:
        #    print i
    return radav

### the SIA function 
def SIA(image, block_size, offset_val):
    ''' Computes image autocorrelation. 
    Takes as input:
        image: 2D image
        filter: Boolean, if true will filter image with uniform filter
        filtersize: size for uniform filtering
    Returns:
        corr_im: the image autocorrelation (this will be same size as image)
        rav_corr: radially averaged image autocorrelation '''
    SIA_details = " (raw images)"
    ### Crop image
    #image = image[:1440, :1440]    ###option to crop out any large noise features 
    if skeletonize == True:
        image = threshold_images(image, block_size, offset_val)
        SIA_details = ", (bsize= "+str(block_size)+", offset= "+str(offset_val)+")"
    
    image = 1.0*image-image.mean() #subtract mean
    image = image/image.std() #normalize by standard deviation
    corr_im = np.real(fftshift(ifft2(fft2(image)*np.conj(fft2(image)))))/(image.shape[0]*image.shape[1])
    ###           ^use "np.real()" instead of "np.abs()" b/c after subtracting mean, at large distances autocorrelation values 
    ###                in the g(r) curve should be equally likely to be either positive or negative (np.abs() is incorrect)
    
    ### radial average taken
    rav_corr = newRadav(corr_im)
    
    ### return ONLY rav_corr **b/c I don't need corr_im** (change this if you do need corr_im) 
    return rav_corr, SIA_details

### the following functions ('show_threshold_images', and 'show_filtered_images') are used to preview 
#           the images intended for analysis
def show_raw_images(ax, i, frame_key):
    #print(num_files_array[i])
    plt.gray()
    if num_files_array[i] == 0:
        test_image = np.zeros((1440,1920))
        ax.set_title('[no image]', fontsize=10)
    else:
        test_image = tifffile.imread(files[i],key=[frame_key])
        title = str((files[i].split('\\')[-1])[:-4]) #OPTION: frame_array[i]
        ax.set_title(title, fontsize=10) 
    ax.imshow(test_image) #cmap = 'gray'
    ax.axis('off')
    plt.tight_layout(pad=.2)
    
def show_threshold_images(ax, i, frame_key, block_size, offset_val):
    if num_files_array[i] == 0:
        threshold_image = np.zeros((1440,1920))
        ax.set_title('[no image]', fontsize=10)
    else:
        raw_image = tifffile.imread(files[i],key=[frame_key])  
        ax.set_title(" --> skeletonized", fontsize=10)
        threshold_image = threshold_images(raw_image, block_size, offset_val)
    ax.imshow(threshold_image)
    ax.axis('off')
    plt.tight_layout(pad=.2)

### the function to actually threshold images 
def threshold_images(image, block_size, offset_val):
    ed_image = (image**0.2)*500    #this seems to help generalize thresholding for my data (idrk why). change/remove as needed
    ed_thresh = threshold_local(ed_image, block_size, offset= offset_val)
    ed_binary_im = 1*(ed_image > ed_thresh)
    binned_ed_binary_im = downscale_local_mean(ed_binary_im, (2,2), cval=1)
    return binned_ed_binary_im

In [3]:
fig_size = 7,7/1.618
###  size of output figures
font_size = 16
### font size
dpi_num = 600
### image quality level (recommendation: 600)

found 25 files
 0 	 N4by3Hr14run1.tif
 1 	 N4by3Hr14run2.tif
 2 	 N4by3Hr14run3.tif
 3 	 N4by3Hr14run4.tif
 4 	 N4by3Hr14run5.tif
 5 	 N4by3Hr1run1.tif
 6 	 N4by3Hr1run2.tif
 7 	 N4by3Hr1run3.tif
 8 	 N4by3Hr1run4.tif
 9 	 N4by3Hr1run5.tif
 10 	 N4by3Hr21run1.tif
 11 	 N4by3Hr21run2.tif
 12 	 N4by3Hr21run3.tif
 13 	 N4by3Hr21run4.tif
 14 	 N4by3Hr21run5.tif
 15 	 N4by3Hr28run1.tif
 16 	 N4by3Hr28run2.tif
 17 	 N4by3Hr28run3.tif
 18 	 N4by3Hr28run4.tif
 19 	 N4by3Hr28run5.tif
 20 	 N4by3Hr7run1.tif
 21 	 N4by3Hr7run2.tif
 22 	 N4by3Hr7run3.tif
 23 	 N4by3Hr7run4.tif
 24 	 N4by3Hr7run5.tif
tiff file dimensions: (750, 750, 3)
number of files: 25
total number of time points: 25
preview images for N4by3Hr1run1, files 0 - 24


<IPython.core.display.Javascript object>


preview images for N4by3Hr1run1, 26 files (raw images)


In [33]:
date = "Valency4by3NewFigV2"
exp = "N4by3Hr1run1"

### "data_dir" is the pathway to the folder holding the tiff files to be analyzed --> change to your folder location
data_dir = "Z:\\Maya N\\analysis\\SIA for sims\\Valency4by3NewFigV2\\Valency4by3NewFigV2\\" 
data_save = "Z:\\Maya N\\analysis\\SIA for sims\\Valency4by3NewFigV2\\Valency4by3NewFigV2\\"
### "plot_saveto" is the pathway to the folder where plots and results will be saved
plot_saveto = data_save

files = glob.glob(data_dir+"*tif") ### this should generate an ordered list of files in "data_dir" which have "_t" in their name
print("found %i files" % len(files))
for i,f in enumerate(files): print (' %i \t %s' % (i, f.split('\\')[-1]))
    
    
    
    
    
    
    
time_array = [1, 7, 14, 21, 28]
### array containing the time points corresponding to consecutive tiff files
### e.g. tiff files with '_t01_' in their filename correspond to t = 1 hour, time_array[0]

frame_names = ["1hr", "7hrs", "14hrs", "21hrs", "28hrs"]
### array containing the name for each frame in a tiff file to be run 

pixel_size = 0.066 #0.0944 #0.09436 #10.597  #0.0944
px = "pix size= " + str(pixel_size)
### pixel size (microns per pixel) of frames/ images in the tiff files --- 40x olympus objective => 0.091 um/px
### IF 2x2 BINNING: multiply the original pixel size by 2^2 = 4, e.g. 4*(0.091 um/px) = 0.364 um/px

eg_im= tifffile.imread(files[0])
print("tiff file dimensions: "+ str(eg_im.shape))

num_files = int((len(files)))
print("number of files: "+ str(num_files))

num_times = num_files         #int((len(files))/3)
print("total number of time points: "+ str(num_times))








### NOTE: this make take a while if you have many and/or large images
### "i" --> this value determines which file index to start at 
i = 0   
num_files = int((len(files)))  ### change "num_files" to a smaller number to skeletonize less images more quickly (e.g. 2)
print("preview images for "+exp+", files "+ str(i) + " - " + str(num_files - 1 + i) )

frame_key = 0 ### "frame_key" specifies which frame of a tiff "tifffile.imread()" will read, i.e. "key = 0" selects frame 1

skeletonize = False ### change this to 'True' to skeletonize images
block_size = 749 ### the number of pixels used for local thresholding, you might try ~half your image size to start
offset_val = -20  ### adjust the threshold according to your needs (e.g. -20 increases the threshold by 20)

### this code sets up the figure for skeletonized images to be shown
num_files_array = [1]* (num_files + i)
if num_files % 2 != 0:
    num_files_array.append(0)
    num_files = num_files + 1
num_rows = int((num_files/5))
if skeletonize == True:
    num_rows = num_rows*2
fig_height = num_rows*2.3
fig, axs = plt.subplots(num_rows, 5, figsize=(10,fig_height))
for j, ax in enumerate(axs.flatten()):
    if skeletonize == True:
        if j % 2 == 0:
            show_raw_images(ax, i, frame_key)
        else:
            show_threshold_images(ax, i, frame_key, block_size, offset_val)
            i = i + 1 
        details = exp+ ", %3.1f files (block_size= %3i, offset= %3i)" %((i+1), block_size, offset_val)
    else:
        show_raw_images(ax, i, frame_key)
        i = i + 1 
        details = exp+ ", %i files (raw images)" %(i+1)
plt.show()

print("preview images for "  + details)
### option to save this figure (uncomment below)
#fig.savefig(plot_saveto + "preview images for " + details + ".jpg", dpi=dpi_num)

running SIA on N4by3Hr14run1, frame 1 ...


<IPython.core.display.Javascript object>


ValueError: `block_size` must be a scalar or have the same length as `image.shape`

In [34]:
expx = "4x3hr14run1x.csv"
expy = "4x3hr14run1y.csv"

time_now = 14
file_index = 0
exp_name = frame_names[2]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr14run2x.csv"
expy = "4x3hr14run2y.csv"

file_index = 1
exp_name = frame_names[2]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr14run3x.csv"
expy = "4x3hr14run3y.csv"

file_index = 2
exp_name = frame_names[2]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr14run4x.csv"
expy = "4x3hr14run4y.csv"

file_index = 3
exp_name = frame_names[2]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr14run5x.csv"
expy = "4x3hr14run5y.csv"

file_index = 4
exp_name = frame_names[2]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr1run1x.csv"
expy = "4x3hr1run1y.csv"

time_now = 1
file_index = 5
exp_name = frame_names[0]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr1run2x.csv"
expy = "4x3hr1run2y.csv"

file_index = 6
exp_name = frame_names[0]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr1run3x.csv"
expy = "4x3hr1run3y.csv"

file_index = 7
exp_name = frame_names[0]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr1run3x.csv"
expy = "4x3hr1run3y.csv"

file_index = 8
exp_name = frame_names[0]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr1run3x.csv"
expy = "4x3hr1run3y.csv"

file_index = 9
exp_name = frame_names[0]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr21run1x.csv"
expy = "4x3hr21run1y.csv"

time_now = 21
file_index = 10
exp_name = frame_names[3]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr21run2x.csv"
expy = "4x3hr21run2y.csv"

file_index = 11
exp_name = frame_names[3]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr21run3x.csv"
expy = "4x3hr21run3y.csv"

file_index = 12
exp_name = frame_names[3]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr21run4x.csv"
expy = "4x3hr21run4y.csv"

file_index = 13
exp_name = frame_names[3]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr21run5x.csv"
expy = "4x3hr21run5y.csv"

file_index = 14
exp_name = frame_names[3]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr28run1x.csv"
expy = "4x3hr28run1y.csv"

time_now = 28
file_index = 15
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr28run2x.csv"
expy = "4x3hr28run2y.csv"

file_index = 16
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr28run3x.csv"
expy = "4x3hr28run3y.csv"

file_index = 17
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr28run4x.csv"
expy = "4x3hr28run4y.csv"

file_index = 18
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr28run5x.csv"
expy = "4x3hr28run5y.csv"

file_index = 19
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr7run1x.csv"
expy = "4x3hr7run1y.csv"

time_now = 7
file_index = 20
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr7run2x.csv"
expy = "4x3hr7run2y.csv"

file_index = 21
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr7run3x.csv"
expy = "4x3hr7run3y.csv"

file_index = 22
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr7run4x.csv"
expy = "4x3hr7run4y.csv"

file_index = 23
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)

In [ ]:
expx = "4x3hr7run5x.csv"
expy = "4x3hr7run5y.csv"

file_index = 24
exp_name = frame_names[4]
frame_key = 0
file_details = str((files[file_index].split('\\')[-1])[:-4]) + ", frame " + str(frame_key +1)
print("running SIA on " + file_details + " ...")





skeletonize = True     #change if desired
block_size = block_size #change if desired
offset_val = offset_val #change if desired

### Set basic plot design 
%matplotlib notebook
fig, ax = plt.subplots(figsize=(fig_size))
ax.tick_params(axis='both', which='major', labelsize=font_size)
markerSize = 10
color = 'red' #change if desired

### "g_of_r_curve" is the g(r) correlation data from SIA analysis, these are discrete data points 
g_of_r_curve = []
image_array = tifffile.imread(files[file_index], key=[frame_key])
g_of_r_curve, SIA_details = SIA(image_array, block_size, offset_val)
    
### corresponding x-values calculated according to the radial average size of the image multipled by "pixel_size" (um/px)
xvalues = np.arange(len(g_of_r_curve))*pixel_size

### plot the results
plt.semilogx(xvalues, g_of_r_curve, '.', ms=markerSize, c=color, label = (str(time_now) + " hrs")) 

print("image size: %2i x %2i pixels" %tuple(image_array.shape))
print("radius of radially averaged image: %5.3f um, last x-value= %5.3f um" %(xvalues[-1], xvalues[-1]))

### legend, labels and title for the plot 
### **note: the title determined in this code block carries over to all other plots unless otherwise specified 
plt.xlabel("Distance ($\mu$m)",fontsize=font_size)
plt.ylabel("Autocorrelation",fontsize=font_size) 
ax.legend(loc=0, markerscale=2.,fontsize=font_size-2)
ax.tick_params(direction='in', which='both', labelsize=font_size)

### set title
title = exp_name + SIA_details #+" no bin "
plt.title(title, fontsize=font_size)
plt.show()








SIA_curve_x = pd.DataFrame(data=xvalues, columns=['Distance (um)'])         #({'Distance (µm)': {xvalues}, 'Autocorrelation': {g_of_r_curve}})
SIA_curve_y = pd.DataFrame(data=g_of_r_curve)
SIA_curve_x.to_csv(expx)
SIA_curve_y.to_csv(expy)

print(SIA_curve_y)